In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

* 改用reuters資料集
* 模型最後的激發函數改為softmax，optimizer=rmsprop，loss=categorical_crossentropy

* 最後train accuracy達0.88，val accuracy約0.7


### 1. 讀入深度學習套件

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import reuters

### 2. 讀入數據

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
len(x_train)

8982

In [ ]:
len(x_test)

2246

In [ ]:
len(x_train[0])

87

In [ ]:
len(x_train[1])

56

In [ ]:
num_classes = max(y_train) + 1
print(num_classes)

46


### 3. 資料處理

In [ ]:
## 為了電腦計算方便 設定每一句話(x)只看100個字
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [ ]:
## 每個數字是一個字
x_train[44]

array([ 626,  450,    8,   16,   40,  129, 2221,    5, 4282,  194,  226,
         31,  332,   21, 2138,  672, 3935, 5853,   24,  144,   41,   30,
       1469, 5226,   22,   10,  322,  164,    4,   78, 1520, 2856,    8,
        194,  269, 1469,  114,   30, 1980,  250, 1119,  238,    9,  299,
         45, 2744,   10, 4103, 2045,  491, 2856,   79, 2818,    2,    6,
          2,   92,    4,  497,   21,  904, 1208,    5,   10,  477, 2266,
        416,   22, 2138,  368,    8, 2138,  138, 4378,    6, 5136, 2138,
        157, 3521, 2856,   24,   16, 1043,  126,   32, 5237,   18,    5,
         25, 1303,   79,   58,   32,   32,   20,    5,  468,  582,   17,
         12], dtype=int32)

In [ ]:
# built in keras function for one-hot encoding
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f'One_hot_train_labels sample : {y_train[0]}')

One_hot_train_labels sample : [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print('Training data shape: ', x_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (8982, 100)
Training labels shape:  (8982, 46)
Test data shape:  (2246, 100)
Test labels shape:  (2246, 46)


### 4. step 01: 打造一個函數學習機

In [ ]:
model = Sequential()

In [ ]:
## 原本產生10000維向量 壓到128維向量
model.add(Embedding(10000, 128))

In [ ]:
## 128可換 dropout(讓大家輪流出現) 避免overfitting
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

In [ ]:
## 0到1之間 用sigmoid
model.add(Dense(46, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 46)                5934      
Total params: 1,417,518
Trainable params: 1,417,518
Non-trainable params: 0
_________________________________________________________________


### 5. step 02: 訓練

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10,
         validation_data=(x_test, y_test))

Epoch 1/10
281/281 [==============================] - 106s 376ms/step - loss: 1.0452 - accuracy: 0.7287 - val_loss: 1.3370 - val_accuracy: 0.6603
Epoch 2/10
281/281 [==============================] - 105s 375ms/step - loss: 0.9433 - accuracy: 0.7575 - val_loss: 1.3398 - val_accuracy: 0.6741
Epoch 3/10
281/281 [==============================] - 104s 369ms/step - loss: 0.8635 - accuracy: 0.7756 - val_loss: 1.2328 - val_accuracy: 0.6910
Epoch 4/10
281/281 [==============================] - 103s 368ms/step - loss: 0.7905 - accuracy: 0.7996 - val_loss: 1.2490 - val_accuracy: 0.6999
Epoch 5/10
281/281 [==============================] - 103s 366ms/step - loss: 0.7309 - accuracy: 0.8160 - val_loss: 1.3527 - val_accuracy: 0.6901
Epoch 6/10
281/281 [==============================] - 103s 365ms/step - loss: 0.6610 - accuracy: 0.8323 - val_loss: 1.2476 - val_accuracy: 0.7053
Epoch 7/10
281/281 [==============================] - 102s 364ms/step - loss: 0.6127 - accuracy: 0.8470 - val_loss: 1.2972 -

### 換個存檔方式

這次是把 model 和訓練權重分開存, 使用上更有彈性。

In [ ]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)
model.save_weights('imdb_model_weights.h5')